In [1]:
import pandas as pd
import sqlalchemy
from math import ceil
import psycopg2
from psycopg2 import extras

In [2]:
try:
    pwd = "xxxxxxxx"
    engine = sqlalchemy.create_engine("postgresql://postgres:{}@xx.xx.xx.xx:5432/postgres".format(pwd))
    
    engine.raw_connection().set_session(autocommit = True)
    
    cur = engine.raw_connection().cursor()
except Exception as e: 
    print("Error trying to connect!")
    print(e)

### Transform `Movies` dataset
- Null values found on these columns, let's define the default value, or just left to null
  + `description` : leave as is (keep null)
  + `metascore` : default to 0
  + `usa_gross_income` : default to `$ 0` (maintain data type consistency as string, so use this instead of numeric 0)
  + `budget` : default to `$ 0` (maintain data type consistency as string, so use this instead of numeric 0)
  + `worlwide_gross_income` : default to `$ 0` (maintain data type consistency as string, so use this instead of numeric 0)
  + `reviews_from_critics` : default to 0
  + `reviews_from_users` : default to 0
  + `production_company` : default to `Unknown`
  + `writer` : default to `Unknown`
  + `language` : default to `Unknown`
  + `director` : default to `Unknown`
  + `actors` : default to `Unknown`
  + `country` : default to `Unknown`

In [3]:
movies = pd.read_sql('stg_movies', engine)

In [4]:
movies.fillna({
    'metascore' : '0', 'usa_gross_income' : '$ 0', 'budget' : '$ 0', 'worlwide_gross_income' : '$ 0', 
    'reviews_from_critics' : '0', 'reviews_from_users' : '0', 'production_company' : 'Unknown', 
    'writer' : 'Unknown', 'language' : 'Unknown', 'director' : 'Unknown', 'actors' : 'Unknown', 
    'country' : 'Unknown'
}, inplace=True)

In [5]:
movies.isnull().sum()

imdb_title_id               0
title                       0
original_title              0
year                        0
date_published              0
genre                       0
duration                    0
country                     0
language                    0
director                    0
writer                      0
production_company          0
actors                      0
description              2115
avg_vote                    0
votes                       0
budget                      0
usa_gross_income            0
worlwide_gross_income       0
metascore                   0
reviews_from_users          0
reviews_from_critics        0
created_date                0
dtype: int64

- Multiple values in cell (based on comma character)
  + `title` : OK to have comma in this field, so it's a single value
  + `original_title` : OK to have comma in this field, so it's a single value
  + `description` : OK to have comma in this field, so it's a single value
  + `genre` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table
  + `country` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table
  + `language` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table
  + `director` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table
  + `writer` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table
  + `production_company` : needs to be separated to different table(s) to achieve 1NF
  + `actors` : needs to be separated to different table(s) to achieve 1NF, will put this on generic lookup table

In [6]:
genre_list = set()
for genre in [x.split(',') for x in movies['genre'].to_list()]:
    genre_list.update(genre)
    

print(genre_list)

{'Animation', 'Adventure', ' Adventure', ' History', ' Reality-TV', ' Biography', 'Family', 'War', 'Sci-Fi', 'Comedy', 'Thriller', ' Family', ' Comedy', ' Crime', 'Crime', 'Fantasy', ' Action', ' Musical', 'Music', 'Action', ' Thriller', ' Horror', ' Film-Noir', ' Sport', 'Horror', 'Adult', 'History', 'Drama', ' Music', 'Western', ' Mystery', 'Documentary', 'Romance', 'Biography', 'Sport', ' Animation', ' News', ' Sci-Fi', 'Film-Noir', 'Musical', ' Drama', ' Romance', ' Western', ' War', ' Fantasy', 'Mystery'}


- `date_published` and year can have different value (year on `date_published` vs year on raw data)

- Need to convert foreign currencies on `budget`, `usa_gross_income`, and `worlwide_gross_income` to **USD**

In [7]:
movies[~movies['budget'].astype(str).str.contains('\$')]['budget'].isnull().sum()

0

In [8]:
exchange_rate_to_usd = {
    "GBP" : 1.39, "INR" : 0.013, "EUR" : 1.19, "AUD" : 0.73, "KRW" : 0.00087, 
    "BRL" : 0.19, "CAD" : 0.8, "NOK" : 0.11, "CZK" : 0.047, "PLN" : 0.26, 
    "HKD" : 0.13, "CLP" : 0.0013, "CNY" : 0.15, "SEK" : 0.12, "NZD" : 0.7, 
    "NGN" : 0.0024, "DKK" : 0.16, "ARS" : 0.01, "HUF" : 0.0033, "MXN" : 0.05, 
    "JPY" : 0.0091, "ILS" : 0.31, "DOP" : 0.018, "ISK" : 0.0081, "DEM" : 0.61, 
    "FRF" : 0.18, "RUR" : 0.014, "ITL" : 0.00061, "BEF" : 0.029, "ESP" : 0.0071, 
    "ATS" : 0.086, "THB" : 0.030, "FIM" : 0.2, "SGD" : 0.74, "CHF" : 1.11
}

def convert_to_usd(amount = "$ 0"):
    if amount[0:1] != "$":
        curr_rate = amount[0:3].strip()
        converted_value = int(amount[3:].strip()) * exchange_rate_to_usd.get(curr_rate, 1)
        return "$ " + str(ceil(converted_value))
    return amount

In [9]:
movies['budget'] = movies['budget'].astype(str).map(convert_to_usd)

In [10]:
movies['usa_gross_income'] = movies['usa_gross_income'].astype(str).map(convert_to_usd)

In [11]:
movies['worlwide_gross_income'] = movies['worlwide_gross_income'].astype(str).map(convert_to_usd)

In [12]:
movies[['budget', 'usa_gross_income', 'worlwide_gross_income']].sample(20)

,budget,usa_gross_income,worlwide_gross_income
27663,$ 0,$ 0,$ 0
46810,$ 0,$ 0,$ 0
67137,$ 0,$ 0,$ 0
49573,$ 0,$ 0,$ 0
31118,$ 65000000,$ 5775076,$ 34227298
56268,$ 0,$ 0,$ 0
29230,$ 0,$ 0,$ 490
58356,$ 0,$ 36986,$ 42781
21795,$ 0,$ 0,$ 0
18610,$ 3000000,$ 4033574,$ 4313829


Now lets convert from string
  - `duration` : integer
  - `avg_vote` : float
  - `metascore` : integer
  - `budget` : integer (remove \\$ sign first)         
  - `usa_gross_income` : integer (remove \\$ sign first)
  - `worlwide_gross_income` : integer (remove \\$ sign first)           

In [13]:
movies[['duration', 'avg_vote', 'metascore', 'budget', 'usa_gross_income', 'worlwide_gross_income']].dtypes

duration                 object
avg_vote                 object
metascore                object
budget                   object
usa_gross_income         object
worlwide_gross_income    object
dtype: object

In [14]:
movies['duration'] = movies['duration'].astype(int)
movies['avg_vote'] = movies['avg_vote'].astype(float)
movies['metascore'] = movies['metascore'].astype(float)

In [15]:
movies[['duration', 'avg_vote', 'metascore', 'budget', 'usa_gross_income', 'worlwide_gross_income']].dtypes

duration                   int64
avg_vote                 float64
metascore                float64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
dtype: object

In [16]:
movies['budget'] = movies['budget'].astype(str).map(lambda amount_val : int(amount_val[1:].strip()))
movies['usa_gross_income'] = movies['usa_gross_income'].astype(str).map(lambda amount_val : int(amount_val[1:].strip()))
movies['worlwide_gross_income'] = movies['worlwide_gross_income'].astype(str).map(lambda amount_val : int(amount_val[1:].strip()))

In [17]:
movies[['duration', 'avg_vote', 'metascore', 'budget', 'usa_gross_income', 'worlwide_gross_income']].dtypes

duration                   int64
avg_vote                 float64
metascore                float64
budget                     int64
usa_gross_income           int64
worlwide_gross_income      int64
dtype: object

- Converting all date_published to date data type, and for the invalid data, give default value to 1-Jan-1900

In [18]:
movies['date_published'] = pd.to_datetime(movies['date_published'], errors='coerce')
movies['date_published'].fillna({'date_published' : '1900-01-01'}, inplace=True)

Transform director, writer and actors from `movies`.  Separate the name into list

In [19]:
column = ['director', 'writer', 'actors']

movies[column]

,director,writer,actors
0,Samuel M. Sherman,"Samuel M. Sherman, Brett Piper","Scott Schwartz, Robert Deveau, Donna Asali, Ro..."
1,Kevin W. Smith,Kevin W. Smith,"Reece Dinsdale, Victoria Smurfit, Clara Bellar..."
2,J.B. Tanko,"Renato Aragão, Luis Bacalov","Renato Aragão, Baiaco, Mário Cardoso, Maria Cl..."
3,Richard Winer,Hans Christian Andersen,"Jay Ripley, Shay Garner, Pat Morrell, Bob O'Co..."
4,Jerald Intrator,"Harold Bonnett, John T. Chapman","Meg Myles, Grayson Hall, Mike Keene, Robert Yu..."
...,...,...,...
85850,Ron Termaat,Ron Termaat,"Viggo Waas, Alice Reys, Rick Engelkes, Victor ..."
85851,"Ere Kokkonen, Spede Pasanen","Ere Kokkonen, Spede Pasanen","Spede Pasanen, Hannes Häyrinen, Simo Salminen,..."
85852,Sohrab Modi,"Rajinder Singh Bedi, Saadat Hassan Manto","Bharat Bhushan, Suraiya, Nigar Sultana, Durga ..."
85853,Andy Milligan,Andy Milligan,"Hal Borske, Carrie Anita, Michael Lunsford, Jo..."


In [20]:
for idx, name in movies.iterrows():
    for col in column:
        movies.at[idx, col] = [x.title().strip() for x in name[col].split(',')]

In [21]:
movies[column]

,director,writer,actors
0,[Samuel M. Sherman],"[Samuel M. Sherman, Brett Piper]","[Scott Schwartz, Robert Deveau, Donna Asali, R..."
1,[Kevin W. Smith],[Kevin W. Smith],"[Reece Dinsdale, Victoria Smurfit, Clara Bella..."
2,[J.B. Tanko],"[Renato Aragão, Luis Bacalov]","[Renato Aragão, Baiaco, Mário Cardoso, Maria C..."
3,[Richard Winer],[Hans Christian Andersen],"[Jay Ripley, Shay Garner, Pat Morrell, Bob O'C..."
4,[Jerald Intrator],"[Harold Bonnett, John T. Chapman]","[Meg Myles, Grayson Hall, Mike Keene, Robert Y..."
...,...,...,...
85850,[Ron Termaat],[Ron Termaat],"[Viggo Waas, Alice Reys, Rick Engelkes, Victor..."
85851,"[Ere Kokkonen, Spede Pasanen]","[Ere Kokkonen, Spede Pasanen]","[Spede Pasanen, Hannes Häyrinen, Simo Salminen..."
85852,[Sohrab Modi],"[Rajinder Singh Bedi, Saadat Hassan Manto]","[Bharat Bhushan, Suraiya, Nigar Sultana, Durga..."
85853,[Andy Milligan],[Andy Milligan],"[Hal Borske, Carrie Anita, Michael Lunsford, J..."


### Transform  `names` dataset

Transform based on previous analysis, into `people` table 

- Null values found on these columns, let's define the default value, or just left to null
    + `reason_of_death` : leave as is (keep null)
    + `place_of_death` : leave as is (keep null)
    + `death_details` : leave as is (keep null)
    + `date_of_death` : leave as is (keep null)
    + `height` : leave as is (keep null)
    + `birth_details` : leave as is (keep null)
    + `date_of_birth` : leave as is (keep null)
    + `bio` : leave as is (keep null)
    + `place_of_birth` : default to `Unknown`
    + `spouses_string` : default to `Unknown`

In [22]:
names = pd.read_sql('stg_names', engine)

In [23]:
names_null = names.isnull().sum() / len(names) * 100
names_null.where(names_null > 0).dropna().sort_values(ascending=False)

reason_of_death    92.377018
place_of_death     87.558825
death_details      86.586386
date_of_death      86.586386
height             84.991518
spouses_string     84.766128
place_of_birth     65.068776
birth_details      62.845098
date_of_birth      62.845098
bio                31.241330
dtype: float64

In [24]:
names.fillna({
    'place_of_birth' : 'Unknown', 'spouses_string' : 'Unknown'
}, inplace=True)

names_null = names.isnull().sum() / len(names) * 100
names_null.where(names_null > 0).dropna().sort_values(ascending=False)

reason_of_death    92.377018
place_of_death     87.558825
death_details      86.586386
date_of_death      86.586386
height             84.991518
birth_details      62.845098
date_of_birth      62.845098
bio                31.241330
dtype: float64

- Multiple values in cell (based on comma character)
    + `name` : OK to have comma in this field, so it's a single value
    + `birth_name` : OK to have comma in this field, so it's a single value
    + `bio` : OK to have comma in this field, so it's a single value
    + `birth_details` : OK to have comma in this field, so it's a single value
    + `place_of_birth` : OK to have comma in this field, so it's a single value
    + `death_details` : OK to have comma in this field, so it's a single value
    + `place_of_death` : OK to have comma in this field, so it's a single value
    + `reason_of_death` : OK to have comma in this field, so it's a single value
    + `spouses_string` : consider as descriptive value in this demo, so it's OK to contains comma (not considered as multiple values)
    + `date_of_birth` : we need to extract year from this field, then just default it to 1-January using extracted year
    + `date_of_death` : we need to extract year from this field, then just default it to 1-January using extracted year

In [25]:
import re

for idx, val in names[names['date_of_birth'].str.contains('[A-Za-z]|^\d{1,4}$', na=False)].iterrows():
    if re.match('^\d{4}', val['date_of_birth'].strip()):
        names.at[idx, 'date_of_birth'] = val['date_of_birth'].strip()[:4] + "-01-01"
    else:
        names.at[idx, 'date_of_birth'] = None

In [26]:
names['date_of_birth'].head(20)

0     1906-10-01
1     1936-11-26
2           None
3     1899-09-03
4     1965-08-07
5     1922-07-11
6           None
7           None
8           None
9           None
10          None
11    1934-07-18
12          None
13          None
14    1971-01-16
15    1907-07-22
16          None
17          None
18    1964-02-25
19    1895-11-18
Name: date_of_birth, dtype: object

In [27]:
import re

for idx, val in names[names['date_of_death'].str.contains('[A-Za-z]|^\d{1,4}$', na=False)].iterrows():
    if re.match('^\d{4}', val['date_of_death'].strip()[:4]):
        names.at[idx, 'date_of_death'] = val['date_of_death'].strip()[:4] + "-01-01"
    else:
        names.at[idx, 'date_of_death'] = None

- A lot of names appears on `movies`, but not on `names`. We must insert those difference into `names`, just use default data for the fields other than name:
    + `imdb_name_id` : use `xx` and 7 digits sequence number
    + `name` : known name from our checking
    + `birth_name` : known name from our checking 
    + `height` : null
    + `bio` : null
    + `birth_details` : null
    + `date_of_birth` : null
    + `place_of_birth` : `Unknown`
    + `death_details` : null
    + `date_of_death` : null
    + `place_of_death` : null
    + `reason_of_death` : null
    + `spouses_string` : `Unknown`
    + `spouses` : null
    + `divorces` : null
    + `spouses_with_children` : null
    + `children` : null

In [28]:
names["spouses"] = names["spouses"].astype(int)
names["divorces"] = names["divorces"].astype(int)
names["spouses_with_children"] = names["spouses_with_children"].astype(int)
names["children"] = names["children"].astype(int)

In [29]:
names.dtypes

imdb_name_id                     object
name                             object
birth_name                       object
height                           object
bio                              object
birth_details                    object
date_of_birth                    object
place_of_birth                   object
death_details                    object
date_of_death                    object
place_of_death                   object
reason_of_death                  object
spouses_string                   object
spouses                           int64
divorces                          int64
spouses_with_children             int64
children                          int64
created_date             datetime64[ns]
dtype: object

### `ratings` analysis

- Null values found on these columns, let's define the default value, or just left to null
    + `females_0age_votes` : default to 0
    + `females_0age_avg_vote` : default to 0
    + `males_0age_votes` : default to 0
    + `males_0age_avg_vote` : default to 0
    + `allgenders_0age_votes` : default to 0
    + `allgenders_0age_avg_vote` : default to 0
    + `females_18age_votes` : default to 0
    + `females_18age_avg_vote` : default to 0
    + `females_45age_votes` : default to 0
    + `females_45age_avg_vote` : default to 0
    + `males_18age_avg_vote` : default to 0
    + `males_18age_votes` : default to 0
    + `females_30age_votes` : default to 0
    + `females_30age_avg_vote` : default to 0
    + `allgenders_18age_votes` : default to 0
    + `allgenders_18age_avg_vote` : default to 0
    + `top1000_voters_votes` : default to 0
    + `top1000_voters_rating` : default to 0
    + `us_voters_votes` : default to 0
    + `us_voters_rating` : default to 0
    + `males_45age_votes` : default to 0
    + `males_45age_avg_vote` : default to 0
    + `females_allages_votes` : default to 0
    + `females_allages_avg_vote` : default to 0
    + `allgenders_45age_votes` : default to 0
    + `allgenders_45age_avg_vote` : default to 0
    + `males_30age_votes` : default to 0
    + `males_30age_avg_vote` : default to 0
    + `allgenders_30age_votes` : default to 0
    + `allgenders_30age_avg_vote` : default to 0
    + `males_allages_votes` : default to 0
    + `males_allages_avg_vote` : default to 0
    + `non_us_voters_rating` : default to 0
    + `non_us_voters_votes` : default to 0

In [30]:
ratings = pd.read_sql('stg_ratings', engine)

In [31]:
r_null = ratings.isnull().sum() / len(ratings) * 100 
r_null.where(r_null > 0).dropna().sort_values(ascending=False)

females_0age_votes           74.239124
females_0age_avg_vote        74.239124
males_0age_votes             68.072914
males_0age_avg_vote          68.072914
allgenders_0age_votes        61.144954
allgenders_0age_avg_vote     61.144954
females_18age_votes           7.595364
females_18age_avg_vote        7.595364
females_45age_votes           3.258983
females_45age_avg_vote        3.258983
males_18age_avg_vote          1.706365
males_18age_votes             1.706365
females_30age_votes           1.099528
females_30age_avg_vote        1.099528
allgenders_18age_votes        0.822317
allgenders_18age_avg_vote     0.822317
top1000_voters_votes          0.790868
top1000_voters_rating         0.790868
us_voters_votes               0.243434
us_voters_rating              0.243434
males_45age_votes             0.117640
males_45age_avg_vote          0.117640
females_allages_votes         0.094345
females_allages_avg_vote      0.094345
allgenders_45age_votes        0.093180
allgenders_45age_avg_vote

In [32]:
ratings.fillna(0, inplace=True)

In [33]:
r_null = ratings.isnull().sum() / len(ratings) * 100 
r_null.where(r_null > 0).dropna().sort_values(ascending=False)

Series([], dtype: float64)

### `title_principals` analysis

- Null values found on these columns, let's define the default value, or just left to null
    + `category` : keep null, a person can only in `category / job / characters`
    + `job` : keep null, a person can only in `category / job / characters`
    + `characters` : keep null, a person can only in `category / job / characters`
- Multiple values in cell (based on comma character)
  + `job` : keep as is, the comma is part of the value
  + `characters` : one person can play multi characters in one movie, so we need to convert this string into list of characters
- Two values missing from `movies` : _'tt1860336', 'tt2082513'_

In [34]:
title_principals = pd.read_sql('stg_title_principals', engine)

In [35]:
title_principals.duplicated(subset=["imdb_title_id", "imdb_name_id", "category", "job", "characters"]).sum()

15

In [36]:
title_principals.drop_duplicates(subset=["imdb_title_id", "imdb_name_id", "category", "job", "characters"], inplace=True)

In [37]:
title_principals.duplicated(subset=["imdb_title_id", "imdb_name_id", "category", "job", "characters"]).sum()

0

In [38]:
pd.api.types.is_string_dtype(title_principals['characters'])

True

In [39]:
for idx, val in title_principals[~title_principals['characters'].isnull()].iterrows():
    title_principals.at[idx, 'characters'] = re.sub('\[|\]|\"', '', val['characters']).split(',')

In [40]:
pd.api.types.is_string_dtype(title_principals['characters'])

True

In [41]:
title_principals.shape[0]

835498

In [42]:
for idx in title_principals[~title_principals['imdb_title_id'].astype(str).str.strip().isin(movies['imdb_title_id'].astype(str).str.strip())].index:
    title_principals.drop(idx, inplace=True)

In [43]:
title_principals.shape[0]

835479

### Loading Data

In [44]:
cur.execute("DROP TABLE IF EXISTS movies CASCADE")

cur.execute("DROP TABLE IF EXISTS movie_numeric_votes CASCADE")

cur.execute("DROP TABLE IF EXISTS movie_avg_votes CASCADE")

cur.execute("DROP TABLE IF EXISTS lookup_hdr CASCADE")

cur.execute("DROP TABLE IF EXISTS lookup_dtl CASCADE")

cur.execute("DROP TABLE IF EXISTS people CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_directors CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_genres CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_countries CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_languages CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_actors CASCADE")

cur.execute("DROP TABLE IF EXISTS movies_writers CASCADE")

cur.execute("DROP TABLE IF EXISTS movie_principal_jobs CASCADE")

cur.execute("DROP TABLE IF EXISTS movie_principal_categories CASCADE")

cur.execute("DROP TABLE IF EXISTS movie_principal_characters CASCADE")

In [45]:
try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies
        (
            movie_id SERIAL,
            title_id VARCHAR NOT NULL,
            title VARCHAR NOT NULL,
            original_title VARCHAR NOT NULL,
            duration_minutes integer NOT NULL DEFAULT 0,
            avg_vote REAL NOT NULL DEFAULT 0,
            count_vote integer NOT NULL DEFAULT 0,
            date_published date,
            production_company VARCHAR,
            description VARCHAR,
            budget_usd bigint NOT NULL DEFAULT 0,
            usa_gross_income_usd bigint NOT NULL DEFAULT 0,
            worldwide_gross_income_usd bigint NOT NULL DEFAULT 0,
            metascore REAL NOT NULL DEFAULT 0,
            reviews_from_users REAL NOT NULL DEFAULT 0,
            reviews_from_critics REAL NOT NULL DEFAULT 0,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (movie_id),
            UNIQUE (title_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movie_numeric_votes
        (
            vote_id serial,
            movie_id integer NOT NULL,
            rating_id integer NOT NULL,
            vote_count integer NOT NULL DEFAULT 0,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (vote_id),
            UNIQUE(movie_id, rating_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movie_avg_votes
        (
            vote_id serial,
            movie_id integer NOT NULL,
            rating_id integer NOT NULL,
            vote_avg REAL NOT NULL DEFAULT 0,
            vote_count integer NOT NULL DEFAULT 0,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (vote_id),
            UNIQUE(movie_id, rating_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS lookup_hdr
        (
            lookup_hdr_id SERIAL,
            lookup_type VARCHAR NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (lookup_hdr_id),
            UNIQUE (lookup_type)
        )
    """)


    cur.execute("""
        CREATE TABLE IF NOT EXISTS lookup_dtl
        (
            lookup_dtl_id SERIAL,
            lookup_hdr_id integer NOT NULL,
            lookup_code VARCHAR NOT NULL,
            lookup_description VARCHAR,
            start_active_date date NOT NULL,
            end_active_date date NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (lookup_dtl_id),
            UNIQUE (lookup_hdr_id,
        lookup_code)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS people
        (
            person_id SERIAL,	
            name_id VARCHAR NOT NULL,
            name VARCHAR NOT NULL,
            birth_name VARCHAR NOT NULL,
            spouse_count SMALLINT,
            divorce_count SMALLINT,
            spouse_with_children_count SMALLINT,
            children SMALLINT,
            spouse_desc VARCHAR,
            bio VARCHAR,
            birth_date date,
            death_date date,
            birth_place VARCHAR,
            death_place VARCHAR,
            death_reason VARCHAR,
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            PRIMARY KEY (person_id),
            unique(name_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_directors
        (
            movie_id integer NOT NULL,
            person_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        person_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_genres
        (
            movie_id integer NOT NULL,
            lookup_dtl_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        lookup_dtl_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_countries
        (
            movie_id integer NOT NULL,
            lookup_dtl_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        lookup_dtl_id)
       )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_languages
        (
            movie_id integer NOT NULL,
            lookup_dtl_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        lookup_dtl_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_writers
        (
            movie_id integer NOT NULL,
            person_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        person_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movies_actors
        (
            movie_id integer NOT NULL,
            person_id integer NOT NULL,
            created_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id,
        person_id)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movie_principal_jobs
        (
            movie_id integer NOT NULL REFERENCES movies,
            person_id integer NOT NULL REFERENCES people,
            job varchar not null,
            ordering SMALLINT default 0,    
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id, person_id, job)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movie_principal_categories
        (
            movie_id integer NOT NULL REFERENCES movies,
            person_id integer NOT NULL REFERENCES people,
            category varchar not null,
            ordering smallint DEFAULT 0,    
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id, person_id, category)
        )
    """)

    cur.execute("""
        CREATE TABLE IF NOT EXISTS movie_principal_characters
        (
            movie_id integer NOT NULL REFERENCES movies,
            person_id integer NOT NULL REFERENCES people,
            character varchar not null,
            ordering smallint DEFAULT 0,    
            created_date TIMESTAMP DEFAULT now(),
            last_updated_date TIMESTAMP DEFAULT now(),
            UNIQUE(movie_id, person_id, character)
        )
    """)
except Exception as e:
    print(e)

In [46]:
try:
    cur.execute("""
        ALTER TABLE lookup_dtl
            ADD FOREIGN KEY (lookup_hdr_id)
            REFERENCES lookup_hdr (lookup_hdr_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_directors
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_directors
            ADD FOREIGN KEY (person_id)
            REFERENCES people (person_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_genres
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_genres
            ADD FOREIGN KEY (lookup_dtl_id)
            REFERENCES lookup_dtl (lookup_dtl_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_countries
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_countries
            ADD FOREIGN KEY (lookup_dtl_id)
            REFERENCES lookup_dtl (lookup_dtl_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_languages
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_languages
            ADD FOREIGN KEY (lookup_dtl_id)
            REFERENCES lookup_dtl (lookup_dtl_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_writers
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_writers
            ADD FOREIGN KEY (person_id)
            REFERENCES people (person_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_actors
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movies_actors
            ADD FOREIGN KEY (person_id)
            REFERENCES people (person_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movie_numeric_votes
            ADD FOREIGN KEY (rating_id)
            REFERENCES lookup_dtl (lookup_dtl_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movie_numeric_votes
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movie_avg_votes
            ADD FOREIGN KEY (rating_id)
            REFERENCES lookup_dtl (lookup_dtl_id)
            NOT VALID
    """)

    cur.execute("""
        ALTER TABLE movie_avg_votes
            ADD FOREIGN KEY (movie_id)
            REFERENCES movies (movie_id)
            NOT VALID
    """)
except Exception as e: 
    print(e)


In [47]:
df_to_movies_table = movies[['imdb_title_id', 'title', 'original_title', 'duration', 'avg_vote', 'votes', 
                            'date_published', 'production_company', 'description', 'budget', 'usa_gross_income',
                            'worlwide_gross_income', 'metascore', 'reviews_from_users', 'reviews_from_critics'
                            ]].rename(columns={
    'imdb_title_id' : 'title_id', 'duration' : 'duration_minutes', 'votes' : 'count_vote', 'budget' : 'budget_usd',
    'usa_gross_income' : 'usa_gross_income_usd', 'worlwide_gross_income' : 'worldwide_gross_income_usd'
})

In [48]:
try:
    sql_df_to_movies = ("""
        INSERT INTO movies (
            title_id, 
            title,
            original_title, 
            duration_minutes,
            avg_vote,
            count_vote,
            date_published,
            production_company,
            description,
            budget_usd,
            usa_gross_income_usd,
            worldwide_gross_income_usd,
            metascore,
            reviews_from_users,
            reviews_from_critics
        ) VALUES (
            %(title_id)s,
            %(title)s,
            %(original_title)s,
            %(duration_minutes)s,
            %(avg_vote)s,
            %(count_vote)s,
            %(date_published)s,
            %(production_company)s,
            %(description)s,
            %(budget_usd)s,
            %(usa_gross_income_usd)s,
            %(worldwide_gross_income_usd)s,
            %(metascore)s,
            %(reviews_from_users)s,
            %(reviews_from_critics)s
        ) ON 
        CONFLICT(title_id) DO 
        UPDATE 
        SET 
            title_id = excluded.title_id,
            title = excluded.title,
            original_title = excluded.original_title,
            duration_minutes = excluded.duration_minutes,
            avg_vote = excluded.avg_vote,
            count_vote = excluded.count_vote,
            date_published = excluded.date_published,
            production_company = excluded.production_company,
            description = excluded.description,
            budget_usd = excluded.budget_usd,
            usa_gross_income_usd = excluded.usa_gross_income_usd,
            worldwide_gross_income_usd = excluded.worldwide_gross_income_usd,
            metascore = excluded.metascore,
            reviews_from_users = excluded.reviews_from_users,
            reviews_from_critics = excluded.reviews_from_critics
    """)
    
    psycopg2.extras.execute_batch(cur, 
                                  sql_df_to_movies, 
                                  df_to_movies_table.transpose().to_dict().values(), 
                                  page_size=500)
except Exception as e:
    print(e)

In [49]:
lookup_type_list = ['country', 'genre', 'language', 'rating']

sql_lookup_hdr = """
    INSERT INTO lookup_hdr
    (
        lookup_type
    ) VALUES 
    (
        %s
    )
"""

In [50]:
try:
    for lkp in lookup_type_list:
        cur.execute(sql_lookup_hdr, (lkp,))
except Exception as e:
    print(e)

In [51]:
sql_lookup_dtl = """
    INSERT INTO lookup_dtl
    (
        lookup_hdr_id,
        lookup_code, 
        lookup_description,
        start_active_date,
        end_active_date
    ) VALUES
    (
        %s,
        %s,
        %s,
        '2021-01-01',
        '2099-12-01'
    ) 
    ON CONFLICT (lookup_hdr_id, lookup_code) 
    DO UPDATE 
    SET 
        lookup_description = excluded.lookup_description
"""

sql_get_lookup_hdr = """
    SELECT lookup_hdr_id 
    FROM lookup_hdr 
    WHERE lower(lookup_type) = lower(%s)
"""

In [52]:
try:
    cur.execute(sql_get_lookup_hdr, ('country',))
    country_hdr_id = cur.fetchone()[0]
    
    list_of_country = set()

    for country_name in movies['country'].astype(str).str.split(','):
        list_of_country.update([name.title().strip() for name in country_name])
        
    for country_name in list_of_country:
        cur.execute(sql_lookup_dtl, (country_hdr_id, country_name.upper(), country_name))
except Exception as e:
    print(e)

In [53]:
try:
    cur.execute(sql_get_lookup_hdr, ('genre', ))
    genre_hdr_id = cur.fetchone()
    
    list_of_genre = set()
    
    for genre_name in movies['genre'].unique():
        list_of_genre.update([name.title().strip() for name in genre_name.split(',')])
    
    for genre in list_of_genre:
        cur.execute(sql_lookup_dtl, (genre_hdr_id, genre.upper(), genre))
except Exception as e:
    print(e)

In [54]:
try:
    cur.execute(sql_get_lookup_hdr, ('language',))
    lang_hdr_id = cur.fetchone()
    
    list_of_lang = set()
    
    for lang in movies['language']:
        list_of_lang.update([name.title().strip() for name in lang.split(',')])
        
    for lang in list_of_lang:
        cur.execute(sql_lookup_dtl, (lang_hdr_id, lang.upper(), lang))
        
except Exception as e:
    print(e)

In [55]:
rating_lookups = [("10", "Score 10"), ("9", "Score 9"), ("8", "Score 8"), ("7", "Score 7"), ("6", "Score 6"),
                ("5", "Score 5"), ("4", "Score 4"), ("3", "Score 3"), ("2", "Score 2"), ("1", "Score 1"),
                ("ALLGENDERS_0AGE" , "All genders age below 18"), ("ALLGENDERS_18AGE" , "All genders age 18 and below 30"),
                ("ALLGENDERS_30AGE" , "All genders age 30 and below 45"), ("ALLGENDERS_45AGE" , "All genders age above 45"),
                ("MALES_ALLAGES" , "Male all ages"), ("MALES_0AGE" , "Male age below 18"),
                ("MALES_18AGE" , "Male age 18 and below 30"), ("MALES_30AGE" , "Male age 30 and below 45"), 
                ("MALES_45AGE" , "Male age above 45"),
                ("FEMALES_ALLAGES" , "Female all ages"), ("FEMALES_0AGE" , "Female age below 18"),
                ("FEMALES_18AGE" , "Female age 18 and below 30"), ("FEMALES_30AGE" , "Female age 30 and below 45"), 
                ("FEMALES_45AGE" , "Female age above 45"),
                ("TOP1000" , "Top 1000 voters"), ("US" , "US voters"), ("NON_US" , "Non US voters")
               ]

try:
    cur.execute(sql_get_lookup_hdr, ('rating', ))
    rating_hdr_id = cur.fetchone()
    
    for rating in rating_lookups:
        cur.execute(sql_lookup_dtl, (rating_hdr_id, rating[0].upper(), rating[1]))
except Exception as e:
    print(e)

In [56]:
df_to_people = names[[
    'imdb_name_id', 'name', 'birth_name', 'spouses', 'divorces', 'spouses_with_children', 'children',
    'spouses_string', 'bio', 'date_of_birth', 'date_of_death', 'place_of_birth', 'place_of_death', 
    'reason_of_death'
]].rename(columns={
    'imdb_name_id' : 'name_id', 'spouses' : 'spouse_count', 'divorces' : 'divorce_count', 
    'spouses_with_children' : 'spouse_with_children_count', 'spouses_string' : 'spouse_desc', 
    'date_of_birth' : 'birth_date', 'date_of_death' : 'death_date', 'place_of_birth' : 'birth_place', 
    'place_of_death' : 'death_place', 'reason_of_death' : 'death_reason'
})

In [57]:
sql_df_to_people = ("""
    INSERT INTO people 
    (
        name_id, 
        name,
        birth_name,
        spouse_count,
        divorce_count,
        spouse_with_children_count,
        children,
        spouse_desc,
        bio,
        birth_date,
        death_date,
        birth_place,
        death_place,
        death_reason
    ) VALUES 
    (
        %(name_id)s, 
        %(name)s,
        %(birth_name)s,
        %(spouse_count)s,
        %(divorce_count)s,
        %(spouse_with_children_count)s,
        %(children)s,
        %(spouse_desc)s,
        %(bio)s,
        %(birth_date)s,
        %(death_date)s,
        %(birth_place)s,
        %(death_place)s,
        %(death_reason)s        
    )
    ON CONFLICT(name_id) 
    DO UPDATE
    SET
        name = excluded.name,
        birth_name = excluded.birth_name,
        spouse_count = excluded.spouse_count,
        divorce_count = excluded.divorce_count,
        spouse_with_children_count = excluded.spouse_with_children_count,
        children = excluded.children,
        spouse_desc = excluded.spouse_desc,
        bio = excluded.bio,
        birth_date = excluded.birth_date,
        death_date = excluded.death_date,
        birth_place = excluded.birth_place,
        death_place = excluded.death_place,
        death_reason = excluded.death_reason
""")

In [58]:
try:
    psycopg2.extras.execute_batch(
        cur,
        sql_df_to_people, 
        df_to_people.transpose().to_dict().values(),
        page_size=500
    )
except Exception as e:
    print(e)

In [59]:
existing_names = set(names["name"].to_list())
list_of_name = set()

columns = ['director', 'writer', 'actors']

for idx, data in movies.iterrows():
    for col in columns: 
        list_of_name.update(data[col])

In [60]:
name_diff = list_of_name - existing_names

In [61]:
sql_non_exist_names = ("""
    INSERT INTO people
    (
        name_id,
        name,
        birth_name,
        birth_place,
        spouse_desc
    ) VALUES
    (
        %s,
        %s,
        %s,
        'Unknown',
        'Unknown'
    ) 
    ON CONFLICT (name_id) 
    DO UPDATE
    SET 
        spouse_count = excluded.spouse_count,
        divorce_count = excluded.divorce_count,
        spouse_with_children_count = excluded.spouse_with_children_count,
        children = excluded.children,
        spouse_desc = excluded.spouse_desc,
        bio = excluded.bio,
        birth_date = excluded.birth_date,
        death_date = excluded.death_date,
        birth_place = excluded.birth_place,
        death_place = excluded.death_place,
        death_reason = excluded.death_reason,
        last_updated_date = now()
""")

try:
    list_of_non_exist_name = [
        (
            'xx{}'.format(re.sub('\W+', '', name.lower())), name, name
        ) for name in name_diff
    ]
    
    psycopg2.extras.execute_batch(
        cur,
        sql_non_exist_names,
        list_of_non_exist_name,
        page_size=500
    )
except Exception as e:
    print(e)

In [62]:
sql_to_movie_directors = ("""
    INSERT INTO movies_directors
    (
        movie_id,
        person_id
    ) VALUES
    (
        %s,
        %s
    )
    ON CONFLICT (movie_id, person_id) 
    DO NOTHING
""")

sql_to_movie_writers = ("""
    INSERT INTO movies_writers
    (
        movie_id,
        person_id
    ) VALUES
    (
        %s,
        %s
    )
    ON CONFLICT (movie_id, person_id) 
    DO NOTHING
""")

sql_to_movie_actors = ("""
    INSERT INTO movies_actors
    (
        movie_id,
        person_id
    ) VALUES
    (
        %s,
        %s
    )
    ON CONFLICT (movie_id, person_id) 
    DO NOTHING
""")

try:
    movie_person_dict = {
        'movie_director_id_list' : [],
        'movie_actors_id_list' : [],
        'movie_writer_id_list' : []
    }
    
    col = ['director', 'writer', 'actors']
    
    cur.execute("SELECT lower(name), person_id FROM people")
    person_dict = dict(cur.fetchall())
    
    cur.execute("SELECT title_id, movie_id FROM movies")
    movie_dict = dict(cur.fetchall())
            
    for idx, val in movies.iterrows():
        for col_type in col:
            for person_name in val[col_type]:
                if movie_dict.get(val['imdb_title_id']) is None or person_dict.get(person_name.lower()) is None: 
                    continue
                else:
                    movie_person_dict.get('movie_{}_id_list'.format(col_type)).append(
                        (
                            movie_dict.get(val['imdb_title_id']), 
                            person_dict.get(person_name.lower())
                        )
                    )
            
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_directors,
        movie_person_dict['movie_director_id_list'],
        page_size=500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_writers,
        movie_person_dict['movie_writer_id_list'],
        page_size=500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_actors,
        movie_person_dict['movie_actors_id_list'],
        page_size=500
    )
        
except Exception as e:
    print(e)

In [63]:
sql_to_movie_genres = ("""
    INSERT INTO movies_genres 
    (
        movie_id,
        lookup_dtl_id
    ) VALUES 
    (
        %s,
        %s
    ) 
    ON CONFLICT(movie_id, lookup_dtl_id) DO NOTHING
""")

sql_to_movie_countries = ("""
    INSERT INTO movies_countries
    (
        movie_id,
        lookup_dtl_id
    ) VALUES 
    (
        %s,
        %s
    )
    ON CONFLICT(movie_id, lookup_dtl_id) DO NOTHING
""")

sql_to_movie_languages = ("""
    INSERT INTO movies_languages 
    (
        movie_id,
        lookup_dtl_id
    ) VALUES 
    (
        %s,
        %s
    )
    ON CONFLICT(movie_id, lookup_dtl_id) DO NOTHING
""")

sql_get_lookup_dtl_id = ("""
    SELECT lookup_code, lookup_dtl_id 
    FROM lookup_dtl 
    WHERE lookup_hdr_id = (
        SELECT lookup_hdr_id 
        FROM lookup_hdr 
        WHERE lower(lookup_type) = lower(%s)
    )
""")

columns = ['genre', 'language', 'country']

movies_lookup_dict = {
    'movie_genre_lookup' : [],
    'movie_language_lookup' : [],
    'movie_country_lookup' : []
}

try:
    cur.execute("SELECT title_id, movie_id FROM movies")
    movie_dict = dict(cur.fetchall())
    
    for col in columns:
        cur.execute(sql_get_lookup_dtl_id, (col,))
        lookup_id_dict = dict(cur.fetchall())
        
        for idx in movies.index:
            for val in movies.iloc[idx][col].split(','):
                movies_lookup_dict.get('movie_{}_lookup'.format(col)).append(
                    (movie_dict.get(movies.iloc[idx]['imdb_title_id']),
                    lookup_id_dict.get(val.strip().upper()))
                )
except Exception as e:
    print(e)

In [64]:
try:
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_genres,
        movies_lookup_dict['movie_genre_lookup'],
        page_size = 500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_languages,
        movies_lookup_dict['movie_language_lookup'],
        page_size = 500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_countries,
        movies_lookup_dict['movie_country_lookup'],
        page_size = 500
    )
except Exception as e:
    print(e)

In [65]:
sql_to_principal_jobs = """
    INSERT INTO
        movie_principal_jobs (
        movie_id,
        person_id,
        job,
        ordering)
    VALUES(
        %s,
        %s,
        %s,
        %s)
    ON CONFLICT (movie_id, person_id, job) 
        DO NOTHING 
"""

sql_to_principal_categories = """
    INSERT INTO
        movie_principal_categories (
        movie_id,
        person_id,
        category,
        ordering)
    VALUES(
        %s,
        %s,
        %s,
        %s)
    ON CONFLICT (movie_id, person_id, category) 
        DO NOTHING """

sql_to_principal_characters = """
    INSERT INTO
        movie_principal_characters (
        movie_id,
        person_id,
        character,
        ordering)
    VALUES(
        %s,
        %s,
        %s,
        %s)
    ON CONFLICT (movie_id, person_id, character) 
        DO NOTHING 
"""

In [66]:
try:
    cur.execute("SELECT lower(name_id), person_id FROM people")
    person_dict = dict(cur.fetchall())
    
    cur.execute("SELECT title_id, movie_id FROM movies")
    movie_dict = dict(cur.fetchall())
    
    movie_principal_cat = []
    movie_principal_char = []
    movie_principal_job = [] 
    
    for idx in title_principals.index:
        movie_id = movie_dict.get(title_principals.loc[idx]['imdb_title_id'])
        person_id = person_dict.get(title_principals.loc[idx]['imdb_name_id'])
        
        if title_principals.loc[idx]['job'] and movie_id and person_id:
            movie_principal_job.append(
                    (movie_id, person_id, 
                    title_principals.loc[idx]['job'],
                    title_principals.loc[idx]['ordering'])
                )
        if title_principals.loc[idx]['category'] and movie_id and person_id:
            movie_principal_cat.append(
                    (movie_id, person_id, 
                    title_principals.loc[idx]['category'],
                    title_principals.loc[idx]['ordering'])
                )
        
        if title_principals.loc[idx]['characters'] and movie_id and person_id:
            movie_principal_char.append(
                    (movie_id, person_id, 
                    title_principals.loc[idx]['characters'],
                    title_principals.loc[idx]['ordering'])
                )

except Exception as e:
    print(e)
    print("Error at index {}".format(idx))

In [67]:
try:
    psycopg2.extras.execute_batch(
        cur,
        sql_to_principal_jobs,
        movie_principal_job,
        page_size=500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_principal_categories,
        movie_principal_cat,
        page_size=500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_principal_characters,
        movie_principal_char,
        page_size=500
    )
except Exception as e:
    print(e)

In [68]:
sql_to_movie_numeric_votes = """
    INSERT INTO movie_numeric_votes (
        movie_id,
        rating_id,
        vote_count)
    VALUES(
        %s,
        %s,
        %s)
    ON CONFLICT (movie_id, rating_id) DO UPDATE
    SET
        vote_count = excluded.vote_count,
        last_updated_date = now()
"""

sql_to_movie_avg_votes = """
    INSERT INTO movie_avg_votes (
        movie_id,
        rating_id,
        vote_avg,
        vote_count)
    VALUES(
        %s,
        %s,
        %s,
        %s)
    ON CONFLICT (movie_id, rating_id) DO UPDATE
    SET
        vote_avg = excluded.vote_avg,
        vote_count = excluded.vote_count
"""

In [71]:
try:
    cur.execute("""SELECT lookup_code, lookup_dtl_id 
                    FROM lookup_dtl 
                    WHERE lookup_hdr_id = (
                    SELECT lookup_hdr_id 
                    FROM lookup_hdr 
                    WHERE lower(lookup_type) = lower(%s))""", 
                    ("rating",)
                )

    rating_lookup_dict = dict(cur.fetchall())
    
    cur.execute("SELECT title_id, movie_id FROM movies")
    movie_dict = dict(cur.fetchall())
    
    movie_num_vote_list = []
    movie_avg_vote_list = []
    
    for idx in ratings.index:
        movie_id = movie_dict.get(ratings.loc[idx]["imdb_title_id"].lower())
        
        movie_num_vote_list.extend(
            [
                (movie_id, rating_lookup_dict.get("10"), ratings.loc[idx]['votes_10']),
                (movie_id, rating_lookup_dict.get("9"), ratings.loc[idx]['votes_9']),
                (movie_id, rating_lookup_dict.get("8"), ratings.loc[idx]['votes_8']),
                (movie_id, rating_lookup_dict.get("7"), ratings.loc[idx]['votes_7']),
                (movie_id, rating_lookup_dict.get("6"), ratings.loc[idx]['votes_6']),
                (movie_id, rating_lookup_dict.get("5"), ratings.loc[idx]['votes_5']),
                (movie_id, rating_lookup_dict.get("4"), ratings.loc[idx]['votes_4']),
                (movie_id, rating_lookup_dict.get("3"), ratings.loc[idx]['votes_3']),
                (movie_id, rating_lookup_dict.get("2"), ratings.loc[idx]['votes_2']),
                (movie_id, rating_lookup_dict.get("1"), ratings.loc[idx]['votes_1'])
            ]
        )
        
        movie_avg_vote_list.extend(
            [
                (movie_id, rating_lookup_dict.get("ALLGENDERS_0AGE"), ratings.loc[idx]["allgenders_0age_avg_vote"], ceil(float(ratings.loc[idx]["allgenders_0age_votes"]))),
                (movie_id, rating_lookup_dict.get("ALLGENDERS_18AGE"), ratings.loc[idx]["allgenders_18age_avg_vote"], ceil(float(ratings.loc[idx]["allgenders_18age_votes"]))),
                (movie_id, rating_lookup_dict.get("ALLGENDERS_30AGE"), ratings.loc[idx]["allgenders_30age_avg_vote"], ceil(float(ratings.loc[idx]["allgenders_30age_votes"]))),
                (movie_id, rating_lookup_dict.get("ALLGENDERS_45AGE"), ratings.loc[idx]["allgenders_45age_avg_vote"], ceil(float(ratings.loc[idx]["allgenders_45age_votes"]))),
                (movie_id, rating_lookup_dict.get("FEMALES_0AGE"), ratings.loc[idx]["females_0age_avg_vote"], ceil(float(ratings.loc[idx]["females_0age_votes"]))),
                (movie_id, rating_lookup_dict.get("FEMALES_18AGE"), ratings.loc[idx]["females_18age_avg_vote"], ceil(float(ratings.loc[idx]["females_18age_votes"]))),
                (movie_id, rating_lookup_dict.get("FEMALES_30AGE"), ratings.loc[idx]["females_30age_avg_vote"], ceil(float(ratings.loc[idx]["females_30age_votes"]))),
                (movie_id, rating_lookup_dict.get("FEMALES_45AGE"), ratings.loc[idx]["females_45age_avg_vote"], ceil(float(ratings.loc[idx]["females_45age_votes"]))),
                (movie_id, rating_lookup_dict.get("FEMALES_ALLAGES"), ratings.loc[idx]["females_allages_avg_vote"], ceil(float(ratings.loc[idx]["females_allages_votes"]))),
                (movie_id, rating_lookup_dict.get("MALES_0AGE"), ratings.loc[idx]["males_0age_avg_vote"], ceil(float(ratings.loc[idx]["males_0age_votes"]))),
                (movie_id, rating_lookup_dict.get("MALES_18AGE"), ratings.loc[idx]["males_18age_avg_vote"], ceil(float(ratings.loc[idx]["males_18age_votes"]))),
                (movie_id, rating_lookup_dict.get("MALES_30AGE"), ratings.loc[idx]["males_30age_avg_vote"], ceil(float(ratings.loc[idx]["males_30age_votes"]))),
                (movie_id, rating_lookup_dict.get("MALES_45AGE"), ratings.loc[idx]["males_45age_avg_vote"], ceil(float(ratings.loc[idx]["males_45age_votes"]))),
                (movie_id, rating_lookup_dict.get("MALES_ALLAGES"), ratings.loc[idx]["males_allages_avg_vote"], ceil(float(ratings.loc[idx]["males_allages_votes"]))),
                (movie_id, rating_lookup_dict.get("TOP1000"), ratings.loc[idx]["top1000_voters_rating"], ceil(float(ratings.loc[idx]["top1000_voters_votes"]))),
                (movie_id, rating_lookup_dict.get("NON_US"), ratings.loc[idx]["non_us_voters_rating"], ceil(float(ratings.loc[idx]["non_us_voters_votes"]))),
                (movie_id, rating_lookup_dict.get("US"), ratings.loc[idx]["us_voters_rating"], ceil(float(ratings.loc[idx]["us_voters_votes"]))),
            ]
        )
except Exception as e:
    print(e)

In [78]:
try:
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_numeric_votes,
        movie_num_vote_list,
        page_size=500
    )
    
    psycopg2.extras.execute_batch(
        cur,
        sql_to_movie_avg_votes,
        movie_avg_vote_list,
        page_size=500
    )
except Exception as e:
    print(e)